# Acoustic Wave Example
This notebook demonstrates how to use SymSolver to work out the dispersion relation for acoustic waves.  
It includes:
- solution to Neutral Acoustic Waves case
- steps for:
  - setting up equations
  - finding the dispersion relation
  - evaluating the dispersion relation (plugging in numbers, converting to a polynomial)
  - plugging in an array, and working with an array of polynomials
- solution to Ion Acoustic Waves case
- telling the code what to do, in case it crashes but you know how to proceed with the algebra...

In [1]:
import SymSolver as ss

## the following lines are helpful while developing SymSolver;
## they let you reload the package without restarting the kernel.
## be careful though, using old SymSolver variables after reloading may cause unexpected behavior.
# ss.enable_reload();
# import SymSolver as ss

import numpy as np  # << for plugging in an array of numbers

import pdb  # (for debugging crashes; after a crash try pdb.pm() in the next cell.)

# Neutral Acoustic Waves

## Define Variables & Equations
you can define variables by hand, or use presets.

In [2]:
# # (OPTION 1) DEFINE BY HAND # #
n, P = ss.symbols('n P')  # ss.symbols(('n', 'P')) is also valid.
u = ss.symbol('u', vector=True)
m = ss.symbol('m', constant=True)  # constant=True --> treat this like a constant (e.g. in derivatives)
gamma = ss.symbol(r'\gamma', constant=True)  # r'\gamma' or '\\gamma'.

# equations - labels are optional, but allow you to do equation_system[label] to get the equation.
continuity = ss.equation(n.dpt() + (n * u).div(), 0, label='continuity')
momentum = ss.equation(n * u.dt_advective(), -P.grad() / m, label='momentum')
adiabatic = ss.equation(P.dt_advective() + gamma * P * u.div(), 0, label='adiabatic')  # adiabatic heating
eqsys = ss.equation_system(continuity, momentum, adiabatic)

In [3]:
# # (OPTION 2) USE PRESETS # #
# first, load presets (only a minimal set of presets are loaded by default)
# loading presets returns a dict and also allows you to get_presets().
# note, you can do ss.load_presets('ALL', dst=locals()) to also load presets into the local namespace.
all_presets = ss.load_presets('ALL')
n, P, u, m, gamma = ss.get_presets('n P u m gamma')  # or ss.get_presets(('n', 'P', 'u', 'm', 'gamma'))
fluid_eqs = ss.get_presets('fluid_eqs')
E, B = ss.get_presets('E B')  # so we can set E and B to 0. E and B appear in the preset fluid_eqs
continuity = fluid_eqs['continuity']
momentum_preset = fluid_eqs['momentum']
momentum = momentum_preset.subs((E, 0), (B, 0)).simplify()  # or .ignore(E, B).simplify()
adiabatic = fluid_eqs['adiabatic']
eqsys = ss.equation_system(continuity, momentum, adiabatic)

### Displaying objects from SymSolver package

In [4]:
### DISPLAY (OPTION 1) - "view"
# Jupyter uses object's _ipython_display_ method to display object.
# For objects in SymSolver, obj._ipython_display_() just calls obj.view().
# Example, you could do eqsys.view(), or just:
eqsys

<IPython.core.display.Math object>

In [5]:
### DISPLAY (OPTION 2) - convert to string
# Converting to string gives a string you can copy-paste directly into LaTeX!
print(eqsys)    # (use str(equations) to get the string instead of printing.)

\begin{align}
(0)    &&      \frac{\partial}{\partial t}(n) + \left( \nabla \right) \cdot \left( n \vec{u} \right) &= 0 \\
(1)    &&      n \left( \frac{\partial}{\partial t}(\vec{u}) + \vec{u} \cdot \left( \nabla \right)(\vec{u}) \right) &= \frac{- \nabla(P)}{m} \\
(2)    &&      \frac{\partial}{\partial t}(P) + \vec{u} \cdot \left( \nabla \right)(P) + \gamma P \left( \nabla \right) \cdot \vec{u} &= 0
\end{align}


In [6]:
### DISPLAY (OPTION 3) - repr
# getting repr shows detailed info about internal contents of the objects.
# for example...
repr(eqsys[0])

"Equation(Sum(DerivativeOperation(DerivativeOperator(DerivativeSymbol(Symbol('t', units_base=[t]), partial=True)), Symbol('n')), OperationDotProduct(VectorDerivativeOperator(VectorDerivativeSymbol(Symbol('x', vector=True, units_base=[L]), partial=True, _nabla=True)), Product(Symbol('n'), Symbol('u', vector=True, units_base=\\frac{[L]}{[t]})))), 0, label='continuity')"

## Applying Linear Theory

In [7]:
# # (OPTION 1) # #
zz2 = eqsys.apply_linear_theory()
zz2

<IPython.core.display.Math object>

In [8]:
# # (OPTION 2) # #
# linearize,
# assume plane waves,
# and assume o0 quantities are constants.
# (see the following few cells..)

In [9]:
## LINEARIZE:
# 1) replace all nonconstant (f) with (f0 + f1)
# 2) assume |f0| >> |f1|
# 3) assume 0'th order equations hold
# 4) neglect any terms which are higher than 1st order, e.g. |f1|^2.
zz0 = eqsys.linearize()
zz0

<IPython.core.display.Math object>

In [10]:
## ASSUME PLANE WAVES:
# 5) assume there exists some k and omega, such that for all nonconstant f:
#    f1 = (f~1) * exp [i (k dot x - omega t) ]
# 6) take any derivatives indicated by the equations
# 7) divide by the common exponent (shared by all terms since all are first order)
# 8) to make equations easier to write, remove all the tildes, i.e. relabel (f~1) --> (f1)
zz1 = zz0.assume_plane_waves()#.simplified()
zz1

<IPython.core.display.Math object>

In [11]:
## ASSUME o0 quantities are constant
# These steps are not inherently required for SymSolver to work.
# However, IF YOU REMOVE THE ASSUMPTIONS, you may get an error.
#    This may be due to not having enough equations to solve for all the unknowns,
#    or due to the equations being in forms that SymSolver doesn't (yet?) know how to solve.

zz2 = zz1
## ASSUME 0th order terms are constant (gradients are 0)
# (Symsolver will not be able to solve this system if you try to keep all the gradients.)
zz2 = zz2.assume_o0_constant().simplify()
zz2

<IPython.core.display.Math object>

## Find the Dispersion Relation

In [12]:
# for the simple example here, we're also going to assume u0 == 0.
# (Symsolver can still solve the system even if u0 != 0. This assumption just makes it simpler.)
zz3 = zz2.ignore(u.o0).simplify()
zz3

<IPython.core.display.Math object>

In [13]:
## FIND THE DISPERSION RELATION
solver = zz3.get_o1_solver()
# you can now either repeatedly call solver.solvestep(), or just call solver.solve() one time.
# solvestep lets you see the results of each step, where an eqn is solved then subbed into the other eqns.
# [TODO] need to update SymSolver to do a better job explaining / showing work when solving.

In [14]:
solver.solvestep()

<IPython.core.display.Math object>

In [15]:
solver.solvestep()

<IPython.core.display.Math object>

In [16]:
disprel = solver.solution
solver.solution

<IPython.core.display.Math object>

In [17]:
## SIMPLIFY THE DISPERSION RELATION
omega = ss.OMEGA   # < the omega which is used in linear theory.
k = ss.K           # < the k which is used in linear theory.
qq0 = disprel.polynomialize(omega).simplify()   # see also: polyfractionize
qq0

<IPython.core.display.Math object>

## Plugging in Numbers
Now that we got a polynomial, we can substitute in some numbers and find the roots

In [18]:
## PLUG IN SOME NUMBERS
substitutions = [
    (P.o0, 1),   # P0 = 1
    (n.o0, 1),   # n0 = 1
    (gamma, 5/3),  # gamma = 5/3
    (m, 1),        # m = 1
]
qq1 = qq0.subs(*substitutions).simplified()  # make the substitutions, and simplify
qq1  # note, the viewing-precision can be controlled by ss.DEFAULTS.STRINGREP_NUMBERS_PRECISION.

<IPython.core.display.Math object>

in order to find roots numerically, we should plug in numbers for all variables except $\omega$. So, we need to choose some values for $k$.

In [19]:
kmags = np.linspace(-6, 6, 13)  # values for |k|
qq2 = qq1.sub(k.mag, kmags)
qq2  # note, arrays are displayed in a pretty way that tells you some info about them.

<IPython.core.display.Math object>

In [20]:
qq3 = qq2.simplify()
qq3

<IPython.core.display.Math object>

## CONVERT TO RATIO OF POLYNOMIALS; SOLVE
In general, any dispersion relation from a set of linear equations can be converted into a ratio of polynomials in $\omega$. Use `.polyfraction(omega)` to convert a SymSolver expression into a ratio of polynomials in `omega`. 

In [21]:
## CONVERT TO RATIO OF POLYNOMIALS  ("POLYFRAC")
pf = qq3.lhs.polyfraction(omega)   # ratio of polynomials in omega
# also try: pf.shrinkdegree().shrinkcoeff() for more complex dispersion relations.
pf

<IPython.core.display.Math object>

In [22]:
## SOLVE:
roots = pf.roots()
roots

array([[-7.74596669,  7.74596669],
       [-6.45497224,  6.45497224],
       [-5.16397779,  5.16397779],
       [-3.87298335,  3.87298335],
       [-2.5819889 ,  2.5819889 ],
       [-1.29099445,  1.29099445],
       [ 0.        ,  0.        ],
       [-1.29099445,  1.29099445],
       [-2.5819889 ,  2.5819889 ],
       [-3.87298335,  3.87298335],
       [-5.16397779,  5.16397779],
       [-6.45497224,  6.45497224],
       [-7.74596669,  7.74596669]])

# Ion Acoustic Waves
demonstrating some slightly-more-complicated algebra, and how to use subscripts. 

## Define Variables & Equations
you can define variables by hand, or use presets.

In [23]:
# # (OPTION 1) DEFINE BY HAND # #
ns, Ps = ss.symbols('n P', ['s'])  # equivalent: ss.symbols('n P', subscripts=['s'])
us = ss.symbol('u', ['s'], vector=True)
ms, qs = ss.symbols('m q', ['s'], constant=True)
gamma = ss.symbol(r'\gamma', constant=True)
E = ss.symbol('E', vector=True)

# equations - labels are optional, but allow you to do equation_system[label] to get the equation.
continuity_s = ss.equation(ns.dpt() + (ns * us).div(), 0, label='continuity')
momentum_s = ss.equation(ns * us.dt_advective('s'), -Ps.grad() / ms + qs * ns * E / ms, label='momentum')
adiabatic_s = ss.equation(Ps.dt_advective('s') + gamma * Ps * us.div(), 0, label='adiabatic')  # adiabatic heating
eqsys_s = ss.equation_system(continuity_s, momentum_s, adiabatic_s)

In [24]:
# # (OPTION 2) USE PRESETS # #
# first, load presets (only a minimal set of presets are loaded by default)
# loading presets returns a dict and also allows you to get_presets().
# note, you can do ss.load_presets('ALL', dst=locals()) to also load presets into the local namespace.
all_presets = ss.load_presets('ALL')
ns, Ps, us, ms, gamma, E, B = ss.get_presets('ns Ps us ms gamma E B')
fluid_eqs_s = ss.get_presets('fluid_eqs_s')
continuity_s = fluid_eqs_s['continuity_s']
momentum_preset_s = fluid_eqs_s['momentum_s']
momentum_s = momentum_preset_s.ignore(B).simplify()  # ignore magnetic field.
adiabatic_s = fluid_eqs_s['adiabatic_s']
eqsys_s = ss.equation_system(continuity_s, momentum_s, adiabatic_s)

In [25]:
eqsys_s

<IPython.core.display.Math object>

### Brief sidenote -- using subscripts

In [26]:
# you can swap subscripts, via obj.subscript_swap(old, new). Alias: 'ss'
# this gives a copy of obj with that subscript replaced:
Ps.subscript_swap('s', 'e').view()
(ns * us).ss('s', 'e').view()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [27]:
# you can also delete subscripts, via obj.del_subscripts(*subscripts). Alias: 'del_ss'
# this gives a copy of obj with that subscript removed:
Ps.del_subscripts('s').view()
(ns * us).del_ss('s').view()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [28]:
# These methods works on more complicated objects too, including full equation systems:
eqsys.ss('s', 'e')

<IPython.core.display.Math object>

### Defining remaining variables

In [29]:
# # (OPTION 1) DEFINE BY HAND # #
objs = [ns, Ps, us, ms, qs, continuity_s, momentum_s, adiabatic_s]
ne, Pe, ue, me, qe, continuity_e, momentum_e, adiabatic_e = [obj.ss('s', 'e') for obj in objs]
ni, Pi, ui, mi, qi, continuity_i, momentum_i, adiabatic_i = [obj.ss('s', 'i') for obj in objs]
emom_no_inertia = momentum_e.ignore(ue.dt_advective('e')).simplify(only='simplify_id')

In [30]:
# # (OPTION 2) USING PRESETS # #
# assumes you ran this code already: ss.load_presets('ALL')
ne, Pe, ue, me, qe, continuity_e, momentum_e, adiabatic_e, \
ni, Pi, ui, mi, qi, continuity_i, momentum_i, adiabatic_i \
= ss.get_presets(('ne', 'Pe', 'ue', 'me', 'qe', 'eq_continuity_e', 'eq_momentum_e', 'eq_adiabatic_e',
                  'ni', 'Pi', 'ui', 'mi', 'qi', 'eq_continuity_i', 'eq_momentum_i', 'eq_adiabatic_i'))
B = ss.get_presets('B')  # so we can say to ignore B; B is in the preset momentum equation.
emom_no_inertia = ss.get_presets('eq_momentum_e_no_inertia').ignore(B).simplify(only='simplify_id')
momentum_i = momentum_i.ignore(B).simplify(only='simplify_id')

In [31]:
# # DEFINE QUASINEUTRALITY EQUATION # #
quasineutral = ss.equation(qe * ne + qi * ni, 0, label='QN')

In [32]:
eqsys = ss.equation_system(continuity_e, continuity_i,
                           emom_no_inertia, momentum_i,
                           adiabatic_e, adiabatic_i,
                           quasineutral)
eqsys

<IPython.core.display.Math object>

## Apply Linear Theory

In [33]:
eqs0 = eqsys.apply_linear_theory()

# also, for simplicity in this example, ignore ue0 and ui0
eqs1 = eqs0.ignore(ue.o0, ui.o0).simplify()
eqs1

<IPython.core.display.Math object>

In [34]:
# also, for simplicity in this example, plug in the zeroth-order quasineutrality equation,
#   ne.o0 = - qi ni.o0 / qe
# note: SymSolver doesn't automatically solve 0th order equations or plug them in for you,
#   but it is happy to do those things if you ask it (explicitly).
qn0 = quasineutral.get_o0().solve(ne.o0)
qn0.view()
print('---')
eqs2 = eqs1.subs(qn0).simplify()
eqs2

<IPython.core.display.Math object>

---


<IPython.core.display.Math object>

## Find the Dispersion Relation

In [35]:
# simplify_mode tells how to simplify after each solvestep.
# 'simplify' means apply simplifying algorithms
# 'expand' means apply expanding algorithms
# 'simplified' means expand then simplify
# tuple means do each in order. E.g. ('expand', 'simplify') equivalent to 'simplified'.
# You might want to play around with different options for different equation systems.
solver = eqs2.get_o1_solver(simplify_mode='simplify')

In [36]:
solver.solvestep()

<IPython.core.display.Math object>

In [37]:
solver.solvestep()

<IPython.core.display.Math object>

In [38]:
solver.solvestep()

<IPython.core.display.Math object>

In [39]:
solver.solvestep()

<IPython.core.display.Math object>

In [40]:
solver.solvestep()

<IPython.core.display.Math object>

In [41]:
solver.solvestep()

<IPython.core.display.Math object>

In [42]:
solver.solvestep()

SolvingPatternError: No solvestep methods worked! Attempted options:
SolveStepInfo(ieqn=5, itarget=0, method=SolverMethodInfo(name='linear_eliminate', ...), ...)
SolveStepInfo(ieqn=5, itarget=0, method=SolverMethodInfo(name='vector_eliminate', ...), ...)

In [43]:
# Oh no, an error!
# (well, SymSolver v1.0.2 gives an error. Maybe it's been fixed since then..)
# This error is because SymSolver wasn't smart enough to try eliminating (k dot ue.o1).
# (It only tried to eliminate ue.o1 but it failed.)
# you can tell it to divide by (k dot ue.o1), though!

# solver.system gets the latest system of equations in the solver object.
eqns = solver.system
eqn = eqns[5]  # [5] since it's the equation labeled [5] which needs solving, above.
# first, let's simplify it a little bit:
eqn = eqn.simplified()
eqn

<IPython.core.display.Math object>

In [44]:
solution = (eqn / k.dot(ue.o1)).simplify()
solution

<IPython.core.display.Math object>

In [45]:
# we can also make the solution a bit prettier.
# SymSolver only knows how to solve linear equations, though.
# But here it is linear in omega^2, so we can do:
result = solution.solve(omega**2)
result

<IPython.core.display.Math object>

In [ ]:
# hurray! That result matches what we get if we calculate the answer by-hand.

In [ ]:
# for a test case, you could try to go back, and remove the "ue.o0==ui.o0==0" assumption we made.
# see if SymSolver can handle that, and if it gets the same answer if you plug in that assumption at the end.
# (I found that it can handle it, but maybe doesn't do as much simplifying as you'd like.
#  Also, it gives the same result after plugging in the assumption at the end!)